<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h1>Notebook 1: Monte Carlo Markov Chains Background</h1>

<h2>Structure</h2>

| Part | Topic | Type |
|:----:|-------|------|
| 1 | Recap: Where We Left Off | Conceptual |
| 2 | Monte Carlo Ingtegration | Conceptual + Code |
| 3 | Markov Chains | Conceptual + Code |
| 4 | The Metropolist Algorithm | Technical |
| - | Exercise: Fitting a Radial Velocity Curve| Hands-on |
| - | Exercise: Increasing `n_obs` | Hands-on |
| - | Proposal Width Tuning | Technical |
| 5 | Detailed Balance & Ergocity | Theoretical |
| 6 | Hamiltonian Monte Carlo & NUTS | Conceptual |
| - | NumPyro Demonstartion | Practical |


<h2>Preamble</h2>

- What this covers
- Won't be required to write code, only run it
- An installation check will be performed
- Reading Guide: Sections marked with "🏛️ **Historical Aside**" are anecdotes and context. They are completely skippable, but can be fun.
</div>


<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h1>Recap: Where We Left Off</h1>

In previous tutorials, we defined Bayes Theorem:
$$P(\theta \mid D) = \frac{\mathcal{L}(\theta) \, \pi(\theta)}{\mathcal{Z}}$$
Recall that each of the are:
- $P(\theta|D)$ - The posterior, i.e. what we believe about our parameters *after* seeing the data.
- $\mathcal{L}(\theta)$ - The likelihood, i.e. the probability of seeing the data given a particular set of parameters (the physics and the noise).
- $\pi(\theta)$ - The prior, i.e. what we believe about our parameters *before* seeing the data
- $\mathcal{Z}$ - The evidence, i.e. the total probability of observing your data, averaged over all parameter values weighted by the prior. In practice, treated as a normalisation constant.

The nice thing about Bayes theorem, is that if we only care about parameter estimation (and not model comparison), we can treat the evidence as a normalisation constant and drop it, leaving:

$$P(\theta \mid D) \propto \mathcal{L}(\theta) \, \pi(\theta)$$

If you do care about model comparison, go back to the Nested Sampling notebook.

In the Bayes 101 notebook, we fit a model to a spectral line by doing a grid search. This model had 4 parameters, we fixed two of them, and tested 250 values each for the other two parameters. This meant, that in total, we tested $250^2 = 62,500$ parameter combinations. If we didn't fix the other two, we would have had to test $250^4 \approx 3.9$ billion parameter combinations. Obviously this is way too many to check. This problem is known as the curse of dimensionality. The basic problem is that high dimensional spaces have a lot of volume, most of which we don't care about. Let's see this in practice by using Monte Carlo methods

-----------

# 🏛️ Historical Aside 1: Monte Carlo
<div style="text-align: center;">
  <img src="Images/MonteCarlo.png" style="max-width: 70%; border-radius: 4px;">
    <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
    Left: A poster depicting the facade of the Monte Carlo Casino at sunset, by Mario Borgoni, c. 1910. Right: 'Souvenir de Monte Carlo' postcard, c. late 1800s, featuring a roulette wheel, and a lady on a donkey, likely a depiction of "lady luck".
  </p>
</div>

### Name:

Monte Carlo is a region within the city state of Monaco (near the French and Italian border). The Casino de Monte-Carlo was opened in 1863 to save the state from bankruptcy. The casino was so successful that income tax in Monaco was abolishd in 1869. It is likely the most famous casino in the world, and is the casino that inspired the casino in the James Bond novel and movie "Casino Royale".

### Connection to Statistics:
Like suprisingly many physics and statistics inventions, Monte Carlo methods origniated during the Manhatten Project (i.e. the nuclear bomb project) at Los Alamos. The Monte Carlo method was invented by Stanislaw Ulam and later formalised with the help of John von Neumann.

#### Stanislaw Ulam

Stanislaw Ulam was a Polish-Born (later American) mathematician. He and his brother left Poland to the USA on a voyage funded by his father and uncle. Eleven days after departure, the Nazis had invaded Poland which they took over within two months. Within two years, most of his family was murdered during the Holocaust. 
In 1943, he (may have) asked John von Neumann (who he had met several years earlier in Warsaw) to find him a war job. In October, he was sent an invitation to join an unidentified project in New Mexico. At the time, he was working at the University of Wisconsin-Madison. As he knew little of New Mexico, he checked out a New Mexico guide book from the university library. In the checkout card inside the guide book, the names of several of his colleagues appeared, all of whom had disappeared to work on the Manhatten Project.

In 1946, he suffered from encephalitis -- a severe, life-threatening inflammation of the brain. This temporarily led to cognitive impairment (difficulty with mathematics, memory issues, etc). During his recovery, he had to relearn basic mathematics. While recovering, he played the solo card game solitare frequently and obsessively. While doing so, he started to wonder about the probability of a successful game. Rather than doing difficult combinatorial analysis, he realised that brute-force random sampling would be easier. This is where the Monte-Carlo method originates from. Allegedly, Nicholas Metropolis coined the name Monte-Carlo after Ulams fathers gambling problem and the name stuck. 

Unlike many of his Manhatten project colleagues, he was neither a nuclear hawk nor dove. Perhaps the reason that he did not become a dove was because the project was ostenstibly started in order to defeat the Nazis (who murdered most of his family), and not the Japanese. He did later support anti-nuclear testing treaties.

----------------

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

# Estimating $\pi$ using Monte Carlo Methods

The core idea is simple: *If computing something analytical is hard, throw random numbers at it and count.*

This is a classic example that uses random sampling to estimate the value of $\pi$. You likely have done this in your first computational physics class. 

Consider a square with a sidelength 2, centred at the origin (so $x,y \in [-1,1]$), with a circled inscribed within it. 
- The area of the square: $A_\text{square} = 2 \times 2 = 4$
- The area of the square: $A_\text{circle} = \pi r^2 = \pi$

The ratio of areas is then:

$$ \frac{A_\text{circle}}{A_\text{square}} = \frac{\pi}{4} $$

We will (uniformly) randomly sample points on a two-dimensional plane with limits of [-1, 1], and we will count the number of points that land within a circle of radius 1. If we assume that the number of points within the circle and outside the circle are proportional to the respective areas (which uniform sampling guarantees in expectation), then the fraction of points landing inside the circle convergess to the fraction of the areas:

$$ \frac{N_\text{inside}}{N_\text{total}} \xrightarrow{N \to \infty} \frac{A_\text{circle}}{A_\text{square}} = \frac{\pi}{4}$$

Rearranging, we get:

$$ \pi \approx 4 \times \frac{N_\text{inside}}{N_\text{total}}$$
</div>

## Define the Geometry

In [ ]:
import numpy as np
import corner
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import gamma
from scipy.stats import chi

In [ ]:
# Draw Circle
t = np.linspace(0,2*np.pi, 360)
circ_x = np.cos(t)
circ_y = np.sin(t)

plt.figure(figsize = (4,4))
plt.plot(circ_x, circ_y, ls = '--')
plt.axis('equal')
plt.xlim([-1, 1])
plt.ylim([-1, 1])

In [ ]:
# Set Seed
np.random.seed(161)

# Define arrays to keep track of things
x_samples = []
y_samples = []
inside_mask = []
running_pi_estimate = []

num_samples = 10000
plot_idxs = [10, 100, 1000, 5000, 10000]


# note this is not an efficient implementation
for i in tqdm(range(num_samples)):
    # draw sample
    x = np.random.uniform(low = -1, high = 1, size = 1)
    y = np.random.uniform(low = -1, high = 1, size = 1)
    x_samples.append(x)
    y_samples.append(y)

    # check if point is within circle
    inside = (x**2 + y**2) <= 1
    inside_mask.append(inside)

    # estimate pi
    pi_estimate = 4 * np.sum(inside_mask) / len(x_samples)
    running_pi_estimate.append(pi_estimate)

    # plot
    if i+1 in plot_idxs:
        x_arr = np.array(x_samples).flatten()
        y_arr = np.array(y_samples).flatten()
        mask = np.array(inside_mask).flatten()
        plt.figure(figsize = (4,4))
        plt.plot(circ_x, circ_y, ls = '--')
        plt.scatter(x_arr[mask], y_arr[mask], color = 'tab:red', s = 2)
        plt.scatter(x_arr[~mask], y_arr[~mask], color = 'black', s = 2)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.xlim([-1.05, 1.05])
        plt.ylim([-1.05, 1.05])  
        plt.title(f'Iterations: {i+1} \n Pi Estimate {pi_estimate}')
        plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">


Let's now track the estimate over time. As $N$ increases, our estimate for $\pi$ should converge to the true value. Since each sample is an independent test, either in or outside the circle (a.k.a. a Bernoulli trial) with success probability $p = \pi/4$, the central limit theorem tells us that the standard error on our estimate is:

$$ \sigma_\pi = 4 \sqrt{\frac{p(1-p)}{N}} \sim \frac{1}{\sqrt{N}}$$

The $\frac{1}{\sqrt{N}}$ scaling is the generic Monte Carlo convergence rate.

</div>

In [ ]:
# Uncertainty estimates
N_arr  = np.arange(1, len(running_pi_estimate) + 1)
pi_est = np.array(running_pi_estimate)
p      = np.pi / 4
envelope = 4 * np.sqrt(p * (1 - p) / N_arr)

# plot
plt.figure(figsize = (15, 10))

plt.subplot(2,1,1)
plt.plot(running_pi_estimate, label = r'$\pi$-estimate')
plt.hlines(np.pi, 0, len(running_pi_estimate), ls = '--', color = 'black', label = r'True $\pi$ value')
plt.legend()
plt.xlim(0, len(running_pi_estimate))
plt.xlabel('Iterations')
plt.ylabel(r'$\pi$ estimate')

plt.subplot(2,1,2)
plt.plot(running_pi_estimate, label = r'$\pi$-estimate')
plt.fill_between(N_arr-1, pi_est - envelope, pi_est + envelope,
                 alpha = 0.2, color = 'steelblue', label = r'$1\sigma$ ($\sim 1/\sqrt{N}$)')

plt.hlines(np.pi, 0, len(running_pi_estimate), ls = '--', color = 'black', label = r'True $\pi$ value')
plt.legend()
plt.xlim(1, len(running_pi_estimate))
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel(r'$\pi$ estimate')

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

## Geometry to Integrals

What we just did is **Monte Carlo integration**. The area of the circle is:

$$A_\text{circle} = \int_{-1}^{1}\int_{-1}^{1} \mathbb{1}[x^2 + y^2 \leq 1] \, dx \, dy$$

where:

$$ \mathbb{1}[x^2 + y^2 \leq 1] = \begin{cases} 1 & \text{if } x^2 + y^2 \leq 1 \\ 0 & \text{otherwise} \end{cases}$$

Rather than analytically calculating the area of a circle, in this case we could have calculated the integral using a grid search. This is fine in two-dimensions, but not practicle in higher dimensions. Instead, we approximated it by sampling:

$$A_\text{circle} \approx \frac{A_\text{square}}{N} \sum_{i=1}^{N} \mathbb{1}[x_i^2 + y_i^2 \leq 1]$$

This is just a fancy way of saying we did random sampling. But the point is that this generalises. For *any* integral $I = \int f(\mathbf{x}) \, d\mathbf{x}$ over some domain, we can estimate it by drawing uniform samples $\mathbf{x}_i$ and computing the average of $f(\mathbf{x}_i)$ scaled by the volume of the domain:


$$I \approx \frac{V}{N} \sum_{i=1}^{N} f(\mathbf{x}_i)$$

Importantly, the convergence rate $\sim 1/\sqrt{N}$ is independent of dimensionality. Grid based numerical integration scales as $N^d$ (The curse of dimensionality). Monte Carlo does not. 

## The Efficiency Catch

In 2D, about $\pi/4 \approx 78.5\%$ of our samples land inside the circle, most of our random throws are "useful". This is rarely the case in higher dimensions. 

Consider the analogous problem: a $d$-dimensional unit hypersphere inscribed in a $d$-dimensional hypercube with side length 2. The volume of a hypercube is:

$$ V_\text{hypercube} = s^d $$

The volume of a hypersphere is:

$$ V_\text{hypersphere} = \frac{\pi^{d/2}}{\Gamma(\frac{d}{2}+1)} r^d$$

Here, $\Gamma$ is [Euler's Gamma function](https://en.wikipedia.org/wiki/Gamma_function). Let's plot the number of useful samples over time (i.e. the samples within the hypersphere)

</div>

In [ ]:
# dimensions
d = np.arange(1, 26)

# Calcalate volumes
hypercube_volume = 2**d
hypersphere_volume = ((np.pi**(d/2)) / (gamma(d/2 + 1))) * 1**d
frac = hypersphere_volume / hypercube_volume


plt.figure(figsize = (6,5))
plt.plot(d, frac, '-o')
plt.xlabel('Dimensions')
plt.ylabel('Volume ratio')
plt.yscale('log')

print(f"d=2:  {frac[1]:.4f}  (1 in ~{1/frac[1]:.1f} samples useful)")
print(f"d=5:  {frac[4]:.4f}  (1 in ~{1/frac[4]:.0f} samples useful)")
print(f"d=10: {frac[9]:.2e}  (1 in ~{1/frac[9]:.0f} samples useful)")
print(f"d=20: {frac[19]:.2e}  (1 in ~{1/frac[19]:.0f} samples useful)")


<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
As dimensionality increases, the sphere occupies a vanishing fraction of the cube. Almost every random sample lands outside the sphere. They are not "wasted" (they still contribute to the estimate), but the signal-to-noise ratio becomes catastrophic. This is the fundamental problem with naive Monte Carlo in high dimensions. The interesting region, whether its a geometeric shape, or the high probability region of a posterior distribution becomes incredibly small. 

This is the problem MCMC solves. Instead of throwing darts uniformly and hoping they land somewhere useful, MCMC constructs a random walk that preferentially visits the interesting region. The samples are no longer independent (they form a correlated chain), and they're no longer uniform (they're drawn from the target distribution$^1$). But they're efficient. Every sample is, in some sense, useful.

*$^1$ We will come back to the importance of these points shortly.*

---------------
</div>


<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h1>Markov Chains</h1>

We have the "Monte Carlo" part. But naive Monte Carlo (throwing darts uniformly) fails in high dimensions because the interesting region is a vanishingly small fraction of the total volume. We need a smarter way to generate random samples, one that concentrates its effort where it matters. This is where Markov Chains come in (the second MC in MCMC).

<h2>A Simple Example: Observing at the AAT</h2>

Those of you who have observed at the Anglo-Australian Telescope at Siding Spring will know that roughly a third of nights are lost to weather (rain, fog, lightning, clouds, etc). But anyone who has actually been on a run knows that bad weather is *not* randomly sprinkled across the schedule. Bad nights come in streaks. If tonight is closed, tomorrow is more likely to be closed too. The same front that shut you down doesn't vanish overnight. Conversely, a clear night is a good omen for the next one.

The overall 33% loss rate is an average over all nights. But the night-to-night weather is **correlated**. 

Let's model AAT weather as a two-state Markov chain: Open and Closed. The weather tomorrow depends only on the weather tonight:

- If tonight is open, there's a 80% chance tomorrow is open and a 20% chance tomorrow is closed.
- If tonight is closed, there's a 40% chance tomorrow is open and a 60% chance tomorrow is closed.

<div style="text-align: center;">
  <img src="Images/MarkovChain.PNG" style="max-width: 70%; border-radius: 4px;">
</div>


We can write this as a transition matrix:

$$T = \begin{pmatrix} 0.80 & 0.20 \\ 0.40 & 0.60 \end{pmatrix}$$

where $T_{ij}$ is the probability of going from state $i$ to state $j$. This is a Markov chain: the weather tomorrow depends *only* on the weather tonight, not on the entire history of the run. This is called the memoryless property. You might see this written as such:

$$P(\theta_{n+1} \mid \theta_n, \theta_{n-1}, \ldots, \theta_0) = P(\theta_{n+1} \mid \theta_n)$$

This just says that the probability of the next state being conditional on all previous states is the same as the probability of the next state being conditional on only the prior state. 

Note: in reality, weather isn't perfectly Markov but it's a decent approximation, and it captures the essential point: the overall rate (33% closed) emerges from correlated night-to-night transitions, not from independent coin flips.

</div>

In [ ]:
# Transition matrix: rows = current state, columns = next state
# State 0 = Open, State 1 = Closed
T = np.array([[0.80, 0.20],
              [0.40, 0.60]])

# Simulate an observing season
np.random.seed(42)
n_nights = 365
weather = np.zeros(n_nights, dtype=int)
weather[0] = 0  # first night is open

for i in range(1, n_nights):
    weather[i] = np.random.choice([0, 1], p=T[weather[i-1]])

# Running fraction of open nights
open_fraction = np.cumsum(weather == 0) / np.arange(1, n_nights + 1)

In [ ]:
# plot schedule

fig, axes = plt.subplots(2, 1, figsize=(14, 6), gridspec_kw={'height_ratios': [1, 2]})

colors = ['tab:blue' if w == 0 else 'tab:orange' for w in weather[:365]]
axes[0].bar(range(365), np.ones(365), color=colors, width=1.0, edgecolor='none')
axes[0].set_xlim(0, 365)
axes[0].set_yticks([])
axes[0].set_xlabel('Night')
axes[0].set_title('green = open, orange = closed — note the streaks')

# Bottom convergence of open fraction
axes[1].plot(open_fraction, color='black', lw=1.5)
axes[1].axhline(2/3, ls='--', color='black', label=f'Stationary: 2/3 ≈ 0.667')
axes[1].set_xlabel('Night')
axes[1].set_ylabel('Fraction of open nights')
axes[1].set_title('Running fraction of open nights')
axes[1].legend()
axes[1].set_xlim(0, n_nights)
axes[1].set_ylim(0.4, 1.0)

plt.tight_layout()
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h2>The Stationary Distribution</h2>

Look at the top panel. The weather comes in streaks, runs of open nights followed by runs of closures. On any given night, the weather is correlated with the night before. Yet, the bottom panel shows that the running fraction of open nights converges to a stable value, regardless of whether the season started clear or cloudy.

This stable value is the **stationary distribution** $\boldsymbol{\pi}^*$. It satisfies:

$$\boldsymbol{\pi}^* = \boldsymbol{\pi}^* T$$

In words: if the chain is currently distributed according to $\boldsymbol{\pi}^*$, then after one transition it's *still* distributed according to $\boldsymbol{\pi}^*$. The distribution is self-perpetuating. So the chain predicts a 33% closure rate, matching the AAT weather statistics.

Let's verify that the chain converges to this regardless of starting conditions:

</div>

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

n_nights = 3*365

for start_state, color, label in [(0, 'tab:blue', 'Start open'), (1, 'tab:orange', 'Start closed')]:
    for trial in range(50):
        weather = np.zeros(n_nights, dtype=int)
        weather[0] = start_state
        for i in range(1, n_nights):
            weather[i] = np.random.choice([0, 1], p=T[weather[i-1]])
        open_frac = np.cumsum(weather == 0) / np.arange(1, n_nights + 1)
        ax.plot(open_frac, color=color, alpha=0.3, lw=1)

    ax.plot([], [], color=color, label=label, lw=2)

ax.axhline(2/3, ls='--', color='black', label='Stationary: 2/3', lw=2)
ax.set_xlabel('Night')
ax.set_ylabel('Fraction of open nights')
ax.set_title('Convergence to stationary distribution — regardless of starting conditions')
ax.legend()
ax.set_xlim(0, n_nights)
ax.set_ylim(0.3, 1.0)
plt.tight_layout()
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h2>Extending to Continuous Distributions</h2>

The AAT weather chain had two states. But most quantities we care about in astronomy are continuous: seeing, flux, temperature, redshift, metaliccity. Can a Markov chain produce a continuous stationary distribution?

Let's stay at the AAT, but instead of tracking whether the dome is open or closed, track the seeing (FWHM in arcseconds) hour by hour through an observing season. The seeing at AAT has a few key properties:
- The median is around 1.5"
- It's positively skewed — you occasionally get terrible 3–4" conditions, but rarely get sub-arcsecond seeing
- It's correlated hour to hour — a period of good seeing is more likely to be followed by another hour of good seeing than a random draw from the overall distribution (similar atmospheric conditions persist)

We can model this as a Markov chain in exactly the same spirit as the weather example. Instead of a transition matrix, we have a transition rule: this hour's seeing depends on the previous hour's seeing, plus some noise, with a tendency to drift back toward the site's typical value. This is an autoregressive process (AR(1)) — "autoregressive" just means "the next value depends on the previous value," which is exactly the Markov property. The "(1)" means it only looks back one step.

The transition rule is:

$$s_{n+1} = \mu + \alpha\,(s_n - \mu) + \varepsilon, \qquad \varepsilon \sim \mathcal{N}(0,\, \sigma^2)$$

where:
- $s_n$ is the (log-)seeing at hour $n$
- $\mu$ is the long-run average log-seeing for the site (i.e. what the seeing "wants" to be)
- $\alpha$ is the **persistence** parameter ($0 \leq \alpha < 1$). High $\alpha$ means the seeing this hour strongly resembles last hour; $\alpha = 0$ means no memory at all
- $\varepsilon$ is a random perturbation, sampled from a normal distribution
- $\sigma$ controls the size of that fluctuation

Read it as: "next hour's seeing = the site average, plus a fraction $\alpha$ of however far the current seeing is from that average, plus random noise." The $\alpha(s_n - \mu)$ term is mean reversion: if the seeing is currently worse than average, it tends to drift back, but not all at once.


</div>

In [ ]:
def simulate_seeing(mu_log, alpha, sigma, n_hours, start=None):
    """Simulate a Markov chain of seeing values (FWHM) via AR(1) in log-space."""
    log_seeing = np.zeros(n_hours)
    log_seeing[0] = start if start is not None else mu_log
    
    for i in range(1, n_hours):
        log_seeing[i] = mu_log + alpha * (log_seeing[i-1] - mu_log) + np.random.normal(0, sigma)
    
    return np.exp(log_seeing)  # convert back to arcseconds

In [ ]:
# AAT: median ~1.5", skewed, moderate persistence
aat_seeing = simulate_seeing(mu_log=np.log(1.5), alpha=0.8, sigma=0.25, n_hours=5000, start=np.log(3.0))

# Mauna Kea: median ~0.6", tighter, similar persistence  
mk_seeing = simulate_seeing(mu_log=np.log(0.6), alpha=0.8, sigma=0.18, n_hours=5000, start=np.log(2.0))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# --- Top row: AAT ---
axes[0, 0].plot(aat_seeing[:500], lw=0.8, color='tab:blue')
axes[0, 0].set_ylabel('Seeing (arcsec)')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_title('AAT — hourly seeing (first 500 hours)')
axes[0, 0].axhline(1.5, ls='--', color='black', alpha=0.5, label='Median (1.5")')
axes[0, 0].legend(fontsize=9)

axes[0, 1].hist(aat_seeing[200:], bins=80, density=True, alpha=0.7, color='tab:blue', label='Chain histogram')
axes[0, 1].axvline(np.median(aat_seeing[200:]), ls='--', color='black', label=f'Median: {np.median(aat_seeing[200:]):.2f}"')
axes[0, 1].set_xlabel('Seeing (arcsec)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('AAT — stationary distribution')
axes[0, 1].set_xlim(0, 5)
axes[0, 1].legend(fontsize=9)

# --- Bottom row: Mauna Kea ---
axes[1, 0].plot(mk_seeing[:500], lw=0.8, color='tab:orange')
axes[1, 0].set_ylabel('Seeing (arcsec)')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_title('Mauna Kea — hourly seeing (first 500 hours)')
axes[1, 0].axhline(0.6, ls='--', color='black', alpha=0.5, label='Median (0.6")')
axes[1, 0].legend(fontsize=9)

axes[1, 1].hist(mk_seeing[200:], bins=80, density=True, alpha=0.7, color='tab:orange', label='Chain histogram')
axes[1, 1].axvline(np.median(mk_seeing[200:]), ls='--', color='black', label=f'Median: {np.median(mk_seeing[200:]):.2f}"')
axes[1, 1].set_xlabel('Seeing (arcsec)')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Mauna Kea — stationary distribution')
axes[1, 1].set_xlim(0, 3)
axes[1, 1].legend(fontsize=9)

plt.tight_layout()
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

The left panels show the trace: the sequence of seeing values hour after hour. Notice the streaks: runs of good seeing and runs of bad seeing, just like the open/closed weather streaks. Each hour's seeing depends on the previous hour. The samples are correlated.

The right panels show the histograms of those chains, i.e. the stationary distributions. Both are continuous, skewed, and positive. They look like real seeing distributions because the same underlying physics (persistent atmospheric conditions with random perturbations) is what produces the Markov property in real weather.

The AAT chain settles into a distribution peaked around 1.5" with a heavy tail toward 4". The Mauna Kea chain settles around 0.6" with a tighter spread. The takeaway is that the shape of the stationary distribution is entirely determined by the transition rules.

Now, notice what we didn't do: we never wrote down the stationary distribution analytically. We never said "the seeing follows a log-normal with these parameters." We defined a transition rule (how this hour relates to the previous hour), ran the chain, and the stationary distribution emerged.

MCMC does this in reverse. We start$^2$ with a target distribution (the posterior $P(\theta|D)$) and we create transition rules such that the chain's stationary distribution is exactly that target. The chain then produces samples from the posterior, and the histogram of those samples converges to the posterior shape.

So, how do we engineer those transition rules?

*$^2$This does not mean that we need to already know the posterior.*

-----------


# 🏛️ Historical Aside 2: Monte Carlo and the Nuclear Bomb

<div style="text-align: center;">
  <img src="Images/Pioneerclub.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
         Foreground: Fremont Street, Las Vegas. Background: The (stemless) mushroom cloud of a nuclear bomb test, likely the priscilla test during operation plumbob. Las Vegas branded itself as the "Up and Atom City". Hotels, including the Pioneer Club, held "Atomic Cocktails" parties, and tourists watched the explosions from hotel balconies or downtown rooftops.
  </p>
</div>


After Stanislaw Ulam came up with Monte Carlo sampling for answering the question about the proportion of solvable solitaire games, he described his idea to John von Neumann. Von Neumann immediately realised its significance for solving a major unsolved problem at Los Alamos: neutron diffusion within fissile material.

The core problem was this. In a nuclear weapon, a neutron enters the bomb core and one of several things happens: it gets absorbed by a nucleus and triggers fission (releasing more neutrons), it gets absorbed without fission, it scatters off a nucleus and changes direction and energy, or it escapes the material entirely. Whether the bomb works (whether you get a sustained chain reaction) depends on the balance of these outcomes averaged over enormous numbers of neutrons, each taking a random walk through the material. Each neutron's path is a stochastic trajectory through 3D space with random scattering angles, random energy losses, and probabilistic absorption. The probability of each event depends on the neutron's energy, the local material composition, and the density, which itself is changing because the material is imploding.

Classically, the physicists would need to solve the [Boltzmann transport equation](https://en.wikipedia.org/wiki/Boltzmann_equation), which governs neutron behaviour in a bomb core. But solving this was analytically intractable for realistic bomb geometries. Yet the physicists did have the necessary nuclear data: mean free paths, scattering cross-sections, energy transfer distributions, etc. What Ulam and von Neumann realised is that you don't need to solve the integral. You can just simulate individual neutrons: pick a starting position and direction, draw random numbers to decide what happens at each step, follow the neutron until it either escapes or gets absorbed, and repeat for thousands of neutrons. The bulk quantities of interest (the neutron multiplication factor, the energy deposition profile, whether a given geometry achieves criticality) emerge as averages over many simulated histories. This is Monte Carlo: each neutron plays a random game, and you estimate the physics by playing the game many times and counting outcomes. 

In 1947, von Neumann wrote a letter containing an 81-step pseudocode which was essentially the first Monte Carlo algorithm designed for electronic computation. He proposed tracking 100 neutrons through 100 collisions each and estimated that it would take 5 hours on ENIAC. In 1948, the first simulations were complete. Crucially, the Monte Carlo approach meant you could change the bomb geometry (make the plutonium pit thicker, add a uranium tamper, change the explosive lens configuration) and rerun the simulation to see how it affected the chain reaction. Before Monte Carlo, these design decisions relied on crude analytical approximations and physical intuition. After Monte Carlo, you could iterate computationally. This is a large part of why the hydrogen bomb program progressed as fast as it did in the early 1950s.

In 1949 in Los Angeles, a symposium on Monte Carlo methods was held. The attendees included von Neumann, Tukey, and Neyman among others. This conference was the first time many of the techniques were formally published, and it led to the adoption of Monte Carlo methods by the wider community.

### John von Neumann

<div style="text-align: center;">
  <img src="Images/poker.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
         Stanislaw Ulam claiming his winnings in a game of poker; Nick Metropolis visible on lower-right edge of photo. Metropolis once described what a triumph it was to win ten dollars from John von Neumann, author of a famous treatise on game theory. He then bought his book for five dollars and pasted the other five inside the cover as a symbol of victory
  </p>
</div>

<hr style="border: none; border-top: 1px solid #ccc; margin: 20px 100px;">
<p style="text-align: center; font-style: italic; margin: 10px 60px;">
"I am thinking about something much more important than bombs; I am thinking about computers."
<br><span style="font-style: normal; font-size: 0.9em;">— von Neumann, in a letter to a colleague during WW2</span>
</p>
<hr style="border: none; border-top: 1px solid #ccc; margin: 20px 100px;">


John was born in Budapest, Hungary, in 1903. He was a true prodigy and was able to memorise and recite very lengthy documents. By 17, while still in high school, he had co-published a paper on Cantor's set theory. After being discouraged from pursuing pure mathematics by his father, he came to a compromise: John would complete a degree in chemical engineering at ETH Zurich while simultaneously completing a PhD in mathematics at the University of Budapest. At age 22 he earned his PhD. He was extremely prolific, writing many papers while still in Europe. In 1930, he started working at Princeton as faculty while younger than 30. Allegedly, he was often mistaken for a graduate student. He would often host parties and was known for flowing cocktails (he drank a lot). Perhaps related, he was also known to be a terrible driver, having at least once crashed his car into a tree. Apparently, an intersection near Princeton was informally named "von Neumann corner" as he frequently crashed there.

He was not the only Hungarian who worked at Los Alamos. In fact there were a surprising number not only from Hungary, but from his high school specifically. Around the time of the Manhattan Project and the following years was the first time that "flying saucers" (or UFOs, or UAPs) were culturally significant. This led to conversations around Los Alamos about them, including one that involved Enrico Fermi coining the Fermi Paradox ("If the universe is so apparently habitable, where are all the aliens?"). An often joked solution was that they were already here and they called themselves Hungarians. This was due to the large number of freakishly smart Hungarians around, including von Neumann, and the (now outdated) belief that the Hungarian language was isolated from any other language.

In 1943, Oppenheimer invited von Neumann to Los Alamos. He was an expert in the non-linear physics of hydrodynamics and shock waves. His principal contribution to the project was to the implosion nuclear bomb ("Fat Man" style) rather than to the gun-type bomb ("Little Boy" style). The implosion type bomb is what ended up being more effective, and is the design of modern nuclear weapons. A problem that was becoming apparent during the Manhattan Project was that gun-type bombs, in which a conventional explosive shoots a bullet piece into a target piece to compress it into supercriticality, did not work for the man-made plutonium they had. The implosion style bomb, in which a spherical piece of plutonium is surrounded by precisely timed explosions in order to compress it into criticality, was being worked on, but many doubted it would work. Von Neumann was a major supporter of the implosion design and worked on it extensively due to his knowledge of hydrodynamics and shock waves. His invention of the explosive lens design was what led to the bomb's success.

On a darker note, von Neumann also sat on the target selection committee. His knowledge of shock wave reflections led to a calculation that optimised explosion height for maximised destruction. An airburst at the right height produces a Mach stem (a reinforced shockwave from the interaction of incident and reflected waves) that dramatically increases destructive potential, but also reduces fallout. He also supported nuking Kyoto, reasoning that the destruction of the cultural capital would be psychologically devastating. This was overruled, and Hiroshima and Nagasaki were selected instead. After WW2, von Neumann consulted for many military agencies in the US. He was also a strong advocate for "preventative" war, in which the USA would attack the Soviets before they obtained nuclear weapons. He was quoted saying "If you say bomb them tomorrow, I say why not today?". He did, however, support Oppenheimer when his patriotism was under attack, and allegedly warned Edward Teller that his colleagues at the Livermore lab were too reactionary.

In 1955, like many of his colleagues, he was diagnosed with cancer, possibly due to witnessing many nuclear explosions. As the cancer spread to his brain, his mental faculties deteriorated and he was moved to a private room after his screams and shouts disturbed other patients. His phones were monitored, as military officials worried that he might accidentally leak nuclear secrets in his delirium. There is no moral to this story.

-----------------

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<div style="text-align: center;">
  <img src="Images/MANIAC2.jpg" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
         Mrs. Lois Cook-Leurgan a computer programmer examines the main arithmetic unit of the MANIAC (Mathematical Analyzer, Numerical Integrator And Computer). The computer played an important part in thermonuclear calculations at the University of California's Los Alamos Scientific Laboratory in New Mexico. This was the first computer that the Metropolis Algorithm was run on.
  </p>
</div>

<h1>Part 3: The Metropolis Algorithm</h1>

To answer the question of how to create transition rules that lead to a stationary distribution which matches the posterior, we can use the Metropolis algorithm.

The algorithm that Metropolis, the Rosenbluths, and the Tellers published in 1953 is surprisingly simple:

1. Start at some position $\theta_0$ in parameter space.
2. Propose a new position by adding a random nudge: $\theta' = \theta_n + \varepsilon$, where $\varepsilon \sim \mathcal{N}(0, \sigma_\text{prop}^2)$.
3. Compute the acceptance ratio:
    $$\alpha = \min\left(1, \;\frac{P(\theta'|D)}{P(\theta_n|D)}\right)$$
5. Accept or reject: draw $u \sim \text{Uniform}(0, 1)$. If $u < \alpha$, move to $\theta'$. Otherwise, stay at $\theta_n$.
6. Repeat from step 2.

That's it, the entire algorithm is: propose, compare, accept or reject.

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h2>The acceptance ratio</h2>

The core of the algorithm is the acceptance ratio:

$$\alpha = \min\left(1, \;\frac{P(\theta'|D)}{P(\theta_n|D)}\right)$$

Let's walk through this term by term:

- $P(\theta_n|D)$ is the posterior density at the current position (where we are now).
- $P(\theta'|D)$ is the posterior density at the proposed position (where we're thinking of moving).
- The ratio $P(\theta'|D) / P(\theta_n|D)$ compares the two. If the ratio is 2, the proposed point is twice as probable as the current point. If it's 0.1, the proposed point is ten times less probable.
- The $\min(1, \ldots)$ caps the ratio at 1, because a probability can't exceed 1.

Note that since $P(\theta|D) \propto \mathcal{L}(\theta)\pi(\theta)$, this ratio is really comparing the product of likelihood and prior at the two points. A proposed point can be accepted because it fits the data better (higher likelihood), because it is more consistent with our prior knowledge (higher prior), or both. The Bayesian comparison is baked into every single step of the chain.

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h2>The three outcomes</h2>

There are three possible outcomes at each step:

1. The proposed point has higher posterior density ($P(\theta'|D) > P(\theta_n|D)$):

The ratio $P(\theta'|D)/P(\theta_n|D)$ is greater than 1, so $\alpha = \min(1, \text{something} > 1) = 1$. We always accept. The chain moves to the new point. This makes sense: if the proposed position is more probable, we should go there.

2. The proposed point has equal posterior density ($P(\theta'|D) = P(\theta_n|D)$):

The ratio is exactly 1, so $\alpha = 1$. We always accept. The chain moves. This also makes sense: there's no reason to prefer one point over another if they're equally probable.

3. The proposed point has lower posterior density ($P(\theta'|D) < P(\theta_n|D)$):

The ratio is between 0 and 1, and we accept with exactly that probability. Concretely: if the proposed point is half as probable as the current point, the ratio is 0.5 and we accept with 50% probability. If it's a hundred times less probable, the ratio is 0.01 and we accept with 1% probability. We draw a uniform random number $u$ between 0 and 1, and accept if $u < \alpha$. Otherwise, we stay put and record the current position again.

This third case is the important one. The chain can move to lower-probability regions, but it does so reluctantly and in proportion to how much worse they are. This is what allows the chain to explore the full shape of the posterior, including its tails and secondary peaks, rather than getting stuck at the single highest point. If we only ever accepted uphill moves (cases 1 and 2), the algorithm would just be an optimiser that finds the MAP. By occasionally accepting downhill moves, it becomes a sampler that maps out the entire distribution.

Below is a visualisation of the three outcomes:

</div>

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

x = np.linspace(-5, 6, 1000)

def density(xv):
    return 0.4 * np.exp(-0.5*((xv-2)/0.8)**2) + 0.6 * np.exp(-0.5*((xv+1)/1.2)**2)

target_vals = density(x)
norm = target_vals.max()
target_vals /= norm

cases = [
    {'title': 'Case 1: Uphill → always accept',
     'tc': -2.8, 'tp': -1.0,
     'result': 'Accept ✓', 'rcol': 'tab:green'},
    {'title': 'Case 2: Equal density → always accept', 
     'tc': -2.5, 'tp': 2.71,
     'result': 'Accept ✓', 'rcol': 'tab:green'},
    {'title': 'Case 3: Downhill → accept with prob. α',
     'tc': 2.0, 'tp': 4.0,
     'result': 'Maybe...', 'rcol': 'tab:orange'},
]

for ax, c in zip(axes, cases):
    tc, tp = c['tc'], c['tp']
    pc = density(tc) / norm
    pp = density(tp) / norm
    
    # Background
    ax.plot(x, target_vals, 'k-', lw=2.2)
    ax.fill_between(x, target_vals, alpha=0.06, color='grey')
    
    # Vertical reference lines
    ax.vlines(tc, 0, pc, color='tab:blue', ls=':', alpha=0.3, lw=1)
    ax.vlines(tp, 0, pp, color=c['rcol'], ls=':', alpha=0.3, lw=1)
    
    # Points (BELOW arrow)
    ax.plot(tc, pc, 'o', color='tab:blue', ms=14, zorder=5, 
            markeredgecolor='white', markeredgewidth=2)
    ax.plot(tp, pp, 's', color=c['rcol'], ms=13, zorder=5, 
            markeredgecolor='white', markeredgewidth=2)
    
    # Curved arrow (ON TOP of points)
    rad = -0.4 if tp > tc else 0.4
    ax.annotate('', 
                xy=(tp, pp + 0.03), 
                xytext=(tc, pc + 0.03),
                arrowprops=dict(arrowstyle='->', color='#333', lw=2,
                               connectionstyle=f'arc3,rad={rad}'),
                zorder=10)
    
    # Labels (ABOVE arrow)
    label_bbox = dict(boxstyle='round,pad=0.15', facecolor='white', 
                      edgecolor='none', alpha=0.9)
    
    ax.text(tc, pc + 0.09, r'$\theta_n$', ha='center', fontsize=14, 
            color='tab:blue', fontweight='bold', zorder=15, bbox=label_bbox)
    
    if pp < 0.1:
        ax.text(tp + 0.35, pp + 0.02, r"$\theta'$", ha='left', fontsize=14,
                color=c['rcol'], fontweight='bold', zorder=15, bbox=label_bbox)
    else:
        ax.text(tp, pp + 0.09, r"$\theta'$", ha='center', fontsize=14, 
                color=c['rcol'], fontweight='bold', zorder=15, bbox=label_bbox)
    
    # Result badge
    ax.text(0.5, 0.94, c['result'], transform=ax.transAxes, ha='center', 
            fontsize=13, fontweight='bold', color=c['rcol'],
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                      edgecolor=c['rcol'], alpha=0.95, lw=1.5))
    
    # Alpha value
    ratio = pp / pc
    ratio_text = r'$\alpha$ = 1' if ratio >= 1.0 else rf'$\alpha$ = {ratio:.2f}'
    ax.text(0.5, 0.83, ratio_text, transform=ax.transAxes, ha='center', 
            fontsize=11, color='#666')
    
    ax.set_title(c['title'], fontsize=11, fontweight='bold', pad=14)
    ax.set_xlabel(r'$\theta$', fontsize=12)
    ax.set_yticks([])
    ax.set_xlim(-5, 5.5)
    ax.set_ylim(-0.02, 1.4)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

axes[0].set_ylabel(r'$P(\theta|D)$', fontsize=12)
plt.tight_layout(w_pad=2.5)
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

<h2>Example: Fitting a Radial Velocity Curve</h2>
We will use a manual implementation of the metropolis algorithm to fit a single planet on a circular orbit to a some radial velocity data from the star.

### Model
$$ v_r(t) = \gamma + K \sin \left(\frac{2\pi t}{P} + \phi \right) $$

We will set the true value and (simple) priors:

| Parameter | Symbol | True Value | Prior | Notes |
|-----------|:------:|------------|-------|-------|
| System velocity | $\gamma$ | 5 m/s | Uniform(-50, 50) | |
| Semi-amplitude  | K | 2 m/s | Uniform(0, 100) | Must be positive |
| Period | P | 30 days | Uniform(0.1, 200) | |
| Phase | $\phi$ | 0.5 rad | Uniform(0, $2\pi$) | |

For the data, we will generate 15 observations, randomly sampled over a 100 day period to mimic a real observing campaign. We will include uncertainties of $\sim1 m/s$, which is typical of a good, but not leading, spectrograph. 

</div>

In [ ]:
np.random.seed(42)

# Get observations
n_obs = 15
t_obs = np.sort(np.random.uniform(0, 100, n_obs))

# STD of Uncertainties, ~1m/s with some spread (differing SNR on different nights)
v_err = np.random.uniform(0.8, 2.5, n_obs)

# True Parameters
gamma_true = 5   # m/s
K_true     = 2  # m/s
P_true     = 30  # days
phi_true   = 0.5 # rads

# Generate data
v_model = gamma_true + K_true * np.sin(2*np.pi*t_obs/P_true + phi_true)
v_obs   = v_model + v_err * np.random.randn(n_obs) # realisation of noise

In [ ]:
# Smooth model curves
t_fine = np.linspace(0, 100, 500)
v_fine = gamma_true + K_true * np.sin(2 * np.pi * t_fine / P_true + phi_true)

phase_fine = np.linspace(0, 1, 500)
v_phase_fine = gamma_true + K_true * np.sin(2 * np.pi * phase_fine + phi_true)

# Phase-fold the observations
phase_obs = ((t_obs / P_true) % 1)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4),
                         gridspec_kw={'width_ratios': [2, 1]})

# Panel 1: Time series
ax = axes[0]
ax.errorbar(t_obs, v_obs, yerr=v_err, fmt='o', color='k',
            ecolor='grey', capsize=2, ms=5, label='Observations')
ax.plot(t_fine, v_fine, color='tab:blue', lw=1.5, alpha=0.7, label='True model')
ax.axhline(gamma_true, color='grey', ls=':', lw=0.8)
ax.set_xlabel('Time [days]')
ax.set_ylabel('Radial velocity [m/s]')
ax.legend(fontsize=9)
ax.set_title('RV Time Series')

# Panel 2: Phase-folded
ax = axes[1]
ax.errorbar(phase_obs, v_obs, yerr=v_err, fmt='o', color='k',
            ecolor='grey', capsize=2, ms=5)
ax.plot(phase_fine, v_phase_fine, color='tab:blue', lw=1.5, alpha=0.7)
ax.axhline(gamma_true, color='grey', ls=':', lw=0.8)
ax.set_xlabel('Phase')
ax.set_ylabel('Radial velocity [m/s]')
ax.set_title('Phase-folded')

plt.tight_layout()
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

We will now write a few functions showing the manual implementation of the Metropolis algorithm. Take your time to read it, so you can verify it follows the steps listed above. We will write 4 functions: `log_likelihood`, `log_prior`, `log_posterior`, and the main implentation `metropolis`.
</div>

In [ ]:
def log_likelihood(theta, t, v_obs, v_err):
    '''
    Gaussian log-likelihood for RV data
    ===================================
    Inputs:
        - theta: array of parameters [gamma, K, P, phi]
        - t    : array of observation times
        - v_obs: array of observed RVs
        - v_err: array of measurement uncertainties
    Output:
        log-likelihood value
    '''
    # extract parameters
    gamma, K, P, phi = theta
    model = gamma + K*np.sin(2*np.pi*t/P +phi)

    # calculate log-like (proportional to chi2)
    log_like = -0.5*np.sum(((v_obs-model)/v_err)**2)
    return log_like

def log_prior(theta):
    '''
    Apply priors on all parameters
    '''
    # extract parameter
    gamma, K, P, phi = theta
    
    # Uniform prior: log-probability is 0 (constant) inside bounds, -inf outside.
    # For non-uniform priors, you'd return the log of the probability density instead.
    if not (-50 < gamma < 50):
        return -np.inf
    if not (0 < K < 100):
        return -np.inf
    if not (1 < P < 200):
        return -np.inf
    if not (0 < phi < 2 * np.pi):
        return -np.inf
    return 0.0

def log_posterior(theta, t, v_obs, v_err):
    '''
    Unnormalised log-post = log_prior + log_likelihood
    '''
    lp = log_prior(theta)

    # check if within prior bounds
    if not np.isfinite(lp):
        return -np.inf
    return lp+log_likelihood(theta, t, v_obs, v_err)

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

Recall that the algorithm is:
1. Start at some position $\theta_0$ in parameter space.
2. Propose a new position by adding a random nudge: $\theta' = \theta_n + \varepsilon$, where $\varepsilon \sim \mathcal{N}(0, \sigma_\text{prop}^2)$.
3. Compute the acceptance ratio:
    $$\alpha = \min\left(1, \;\frac{P(\theta'|D)}{P(\theta_n|D)}\right)$$
5. Accept or reject: draw $u \sim \text{Uniform}(0, 1)$. If $u < \alpha$, move to $\theta'$. Otherwise, stay at $\theta_n$.
6. Repeat from step 2.

</div>

In [ ]:
def metropolis(log_post, theta0, proposal_sigma, n_steps, args=()):
    '''
    Metropolis sampler with Gaussian proposals
    ==========================================
    Inputs:
        - log_post       : function
        - theta0         : initial parameters
        - proposal_sigma : array, std of Gaussian proposal per dimension
        - n_steps        : number of iterations
        - args           : tuple of arguments to pass to log_post
    Outputs:
        - chain           : array of shape (n_steps + 1, n_dim)
        - acceptance_rate : float
    '''
    n_dim = len(theta0)
    chain = np.zeros((n_steps + 1, n_dim))
    chain[0] = theta0
    
    log_p_current = log_post(theta0, *args)
    n_accepted = 0

    for i in tqdm(range(n_steps)):
        # propose new point
        theta_prop = chain[i] + proposal_sigma * np.random.randn(n_dim)
        
        # calculate probability at proposed point
        log_p_prop = log_post(theta_prop, *args)
        
        # acceptance ratio
        log_alpha = log_p_prop - log_p_current

        if np.log(np.random.uniform()) < log_alpha:
            # accept: move to proposed point
            chain[i+1] = theta_prop
            log_p_current = log_p_prop
            n_accepted += 1
        else:
            # reject: REPEAT the current position
            chain[i+1] = chain[i]

    acceptance_rate = n_accepted / n_steps
    return chain, acceptance_rate

<div style="max-width: 800px; margin: 0 auto; text-align: justify; padding: 20px; background-color: rgba(0, 64, 133, 0.08); border: 2px solid rgba(0, 64, 133, 0.4); border-radius: 10px;">

<h3>Exercise: Map the Code to the Algorithm</h3>

For each step of the Metropolis algorithm, identify which line(s) of the implementation above correspond to it. Try to answer before revealing.

<b>Step 1:</b> Start at some position $\theta_0$

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

<code>chain.append(theta0)</code> and <code>log_p_current = log_post(theta0, *args)</code>

We store the starting point and evaluate the posterior there so we have something to compare against.

</div>
</details>

<b>Step 2:</b> Propose a new position $\theta' = \theta_n + \varepsilon$, where $\varepsilon \sim \mathcal{N}(0, \sigma_\text{prop}^2)$

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

<code>theta_prop = chain[-1] + proposal_sigma * np.random.randn(n_dim)</code>

<code>chain[-1]</code> is $\theta_n$, and <code>proposal_sigma * np.random.randn(n_dim)</code> is $\varepsilon$.

</div>
</details>

<b>Step 3:</b> Compute the acceptance ratio $\alpha = \min\left(1, \; P(\theta'|D) / P(\theta_n|D)\right)$

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

<code>log_p_prop = log_post(theta_prop, *args)</code><br>
<code>log_alpha = log_p_prop - log_p_current</code>

We work in log-space, so the ratio $P(\theta'|D) / P(\theta_n|D)$ becomes a subtraction: $\log\alpha = \log P(\theta'|D) - \log P(\theta_n|D)$. The $\min(1, \cdot)$ is handled implicitly by the accept/reject step.

</div>
</details>

<b>Step 4:</b> Accept or reject: draw $u \sim \text{Uniform}(0, 1)$. If $u < \alpha$, move to $\theta'$. Otherwise, stay at $\theta_n$.

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

<code>if np.log(np.random.uniform()) < log_alpha:</code><br>
&emsp;&emsp;<code>chain.append(theta_prop)</code><br>
&emsp;&emsp;<code>log_p_current = log_p_prop</code>

Instead of comparing $u < \alpha$, we compare $\log u < \log \alpha$. This avoids exponentiating the log-posterior back to real space, which can overflow. Note that if the proposal is rejected, we don't append anything — the chain just doesn't grow, and <code>chain[-1]</code> remains the current position for the next iteration.

</div>
</details>

<b>Step 5:</b> Repeat from step 2.

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

<code>while len(chain) < n_steps:</code>

The loop continues until we have collected <code>n_steps</code> accepted samples.

</div>
</details>

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
Let's now run the sampler using our data generated above

</div>

In [ ]:
np.random.seed(0)

# Starting point (deliberately offset from truth)
theta0 = np.array([0, 15, 20, 1.0])

# Proposal widths. these matter a lot, and we'll show why
proposal_sigma = np.array([1.0, 1.0, 4.0, 0.3])

# Run
n_steps = 50000
chain, acc_rate = metropolis(
    log_posterior, theta0, proposal_sigma, n_steps,
    args=(t_obs, v_obs, v_err)
)
print(f"Acceptance rate: {acc_rate:.2%}")

In [ ]:
plt.figure(figsize = (10, 10))

plt.subplot(4,1,1)
plt.plot(chain[:,0], lw = 0.5, label = 'Chain')
plt.ylabel(r'System Velocity $\gamma$')
plt.xticks([])
plt.xlim(0, n_steps)
plt.hlines(gamma_true, 0, n_steps, ls ='--', color = 'black', label = 'True Value', alpha = 0.5)
plt.legend()

plt.subplot(4,1,2)
plt.plot(chain[:,1], lw = 0.5, label = 'Chain')
plt.ylabel('Semi Amplitude K')
plt.xticks([])
plt.xlim(0, n_steps)
plt.hlines(K_true, 0, n_steps, ls ='--', color = 'black', label = 'True Value', alpha = 0.5)
plt.legend()

plt.subplot(4,1,3)
plt.plot(chain[:,2], lw = 0.5, label = 'Chain')
plt.hlines(P_true, 0, n_steps, ls ='--', color = 'black', label = 'True Value', alpha = 0.5)
plt.ylabel('Period P')
plt.xticks([])
plt.xlim(0, n_steps)
plt.legend()

plt.subplot(4,1,4)
plt.plot(chain[:,3], lw = 0.5, label = 'Chain')
plt.hlines(phi_true, 0, n_steps, ls ='--', color = 'black', label = 'True Value', alpha = 0.5)
plt.ylabel(r'Phase $\phi$')
plt.xlim(0, n_steps)
plt.xticks([])
plt.legend()
plt.xlabel('Iteration')
plt.xlim(0, n_steps)
plt.tight_layout()

<div style="max-width: 800px; margin: 0 auto; text-align: justify; padding: 20px; background-color: rgba(0, 64, 133, 0.08); border: 2px solid rgba(0, 64, 133, 0.4); border-radius: 10px;">

<b>Question 1:</b> Can you identify the burn in?

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">

The burn-in in this case is very short, only around 10 steps. You can see it as the rapid change in values at the start of the chains.
</div>
</details>

<b>Question 2:</b> What is happening in the Period chain?

<details style="margin: 10px 0;">
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">Click to reveal</summary>

<div style="margin-top: 10px; padding: 15px; background-color: rgba(0, 64, 133, 0.12); border: 1px solid rgba(0, 64, 133, 0.4); border-radius: 5px;">
    Notice that it is jumping from ~30 to ~20 and ~60. These are close to integer ratios. Essentially you are seeing aliasing.
</div>

</div>

In [ ]:
labels = [r'$\gamma$', r'$K$', r'$P$', r'$\phi$']
truths = [gamma_true, K_true, P_true, phi_true]

fig = corner.corner(chain, labels=labels, truths=truths, 
                    truth_color='black', show_titles=True,
                    title_kwargs={"fontsize": 12})

In [ ]:
# Smooth model curves
t_fine = np.linspace(0, 100, 500)
v_fine = gamma_true + K_true * np.sin(2 * np.pi * t_fine / P_true + phi_true)
phase_fine = np.linspace(0, 1, 500)
v_phase_fine = gamma_true + K_true * np.sin(2 * np.pi * phase_fine + phi_true)
# Phase-fold the observations
phase_obs = ((t_obs / P_true) % 1)

# MLE: the sample with the highest log-posterior
log_posts = np.array([log_posterior(chain[i], t_obs, v_obs, v_err) for i in range(len(chain))])
mle = chain[np.argmax(log_posts)]
v_mle = mle[0] + mle[1] * np.sin(2 * np.pi * t_fine / mle[2] + mle[3])
v_mle_phase = mle[0] + mle[1] * np.sin(2 * np.pi * phase_fine + mle[3])

# 100 random posterior samples
idx = np.random.choice(len(chain), 100, replace=False)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4),
                         gridspec_kw={'width_ratios': [2, 1]})

# Panel 1: Time series
ax = axes[0]
for i in idx:
    g, K, P, phi = chain[i]
    ax.plot(t_fine, g + K * np.sin(2 * np.pi * t_fine / P + phi),
            color='black', lw=0.3, alpha=0.15)
ax.plot([], [], color='black', lw=1, alpha=0.5, label='Posterior samples')  # dummy for legend
ax.plot(t_fine, v_mle, color='tab:red', lw=2, ls='--', label='MAP estimate')
ax.errorbar(t_obs, v_obs, yerr=v_err, fmt='o', color='k',
            ecolor='grey', capsize=2, ms=5, label='Observations', zorder=5)
ax.plot(t_fine, v_fine, color='tab:blue', lw=1.5, alpha=0.7, label='True model')
ax.axhline(gamma_true, color='grey', ls=':', lw=0.8)
ax.set_xlabel('Time [days]')
ax.set_ylabel('Radial velocity [m/s]')
ax.legend(fontsize=9)
ax.set_title('RV Time Series')

# Panel 2: Phase-folded
ax = axes[1]
for i in idx:
    g, K, P, phi = chain[i]
    ax.plot(phase_fine, g + K * np.sin(2 * np.pi * phase_fine + phi),
            color='tab:orange', lw=0.3, alpha=0.15)
ax.plot(phase_fine, v_mle_phase, color='tab:red', lw=2, ls='--')
ax.errorbar(phase_obs, v_obs, yerr=v_err, fmt='o', color='k',
            ecolor='grey', capsize=2, ms=5, zorder=5)
ax.plot(phase_fine, v_phase_fine, color='tab:blue', lw=1.5, alpha=0.7)
ax.axhline(gamma_true, color='grey', ls=':', lw=0.8)
ax.set_xlabel('Phase')
ax.set_ylabel('Radial velocity [m/s]')
ax.set_title('Phase-folded')

plt.tight_layout()
plt.show()

<div style="max-width: 800px; margin: 0 auto; text-align: justify; padding: 20px; background-color: rgba(0, 64, 133, 0.08); border: 2px solid rgba(0, 64, 133, 0.4); border-radius: 10px;">

<h3>Exercise: How does more data change the posterior?</h3>

With 15 observations over 100 days, we saw that the posterior had a somewhat significant width, and the period showed aliasing at integer ratios of the true period. Let us know see what happens as we collect more data.

**Task:** Regenerate the observe data `n_obs = 25` and then `n_obs = 100`, keeping everthing else the same. For each case:

1) Run the Metropolis sampler
2) Plot the trace plots
3) Make a corner plot
4) Plot the posterior samples over the data (as above)

**Ask yourself the following:**
- How does the width of each marginal posterior change?
- Does the period aliasing dissapear? Why?
- Does the MAP (Maximum A Posteriori) get closer to the truth?


</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

The proposal standard deviation $\sigma_\text{prop}$ is one of the most important tuning parameters in the Metropolis algorithm. We initally used $\sigma_\text{prop} = [1.0, 1.0, 4.0, 0.3]$ above, and you should have gotten an acceptance rate of $3.2%$. It is low, but not awfully so. 

If your proposals are too small (i.e $\sigma_\text{prop}$ are small), then each proposed step is only a small pertubation from the current position. This small perturbation will likely land at a very similar posterior density, so the acceptance ratio will be close to 1, meaning the step is almost always accepted. This might sound good, but it means the chain is taking baby steps, and isn't exploring the posterior effectively as it basically just hanging around the same spot. 

If $\sigma_\text{prop}$ is large, each proposed step jumps far from the current position. In a typical posterior, most of the parameter space has negligible propability, so a large random jump almost certainly lands somewhere terrible. This means that the acceptance ratio is tiny, the step is rejected, and the chain stays where it is. 

You want something in between: large enough to cover ground, but not so large that they overshoot into unimportant spaces.

### Case 1: Proposals too small

Go back and ensure that the number of observations is reset to 15. Now instead of $\sigma_\text{prop} = [1.0, 1.0, 4.0, 0.3]$, lower it to $\sigma_\text{prop} = [0.1, 0.1, 0.2, 0.01]$. What you should see, now that the proposal width is small relative to the posterior width, is that almost every proposal lands near the current position and is thus accepted. You should have an acceptance rate of $>50%$. This sounds good, but look at the trace plots. You should see that the period stays around 20 days, rather than 30 days. This is because it is incredibly unlikely that a proposed point will stray far from the current position (of around 20 days). 

Another way to think of this is that all of the samples are very highly correlated. This means that the effective sample size is far less than the actual number of samples. This is the slow diffusion failure mode. 

### Case 2: Proposals too large
Set $\sigma_\text{prop} = [10, 10, 25, 1.5]$ and rerun the above code. As the proposal width is very large, the proposed points almost always land in regions of negligible posterior density and are rejected. You should observe an acceptance rate of $\sim 0.03%$, and you should notice it takes much longer to execute. While it does explore parameter space, it is wasting a lot of computational cycles: many points are being proposed, evalutated, and rejected for no gain. While this is not ideal, we do get a better posterior than the too small case. Typically, you want an acceptance rate of ~1-25%. 

## Practical Notes

### Initialisation
**Where should you start you chain?** Hogg & Foreman-Mackey (2018) recommend initialising near a reasonable estimate. For example, near the maximum likelihood solution, or near a value you expect from your prior knowledge. You can initialise randomly within the prior, but this wastes computational cycles on burn-in. 

**If you are concerend that your posterior might be multimodal**, initialise multiple chains at different random locations. If they all converge to the same place that is good. If they don't, then you need to take some care. See the discussion on multi-modality in the main tutorial.

### Thinning
You will sometimes see recommendations to "thin" your chain. This means keep only every $k$-th sample, discarding the rest. The only reason why you would do this is for storage purposes. It throws away valid (if correlated) information. 

### A note on `emcee`
In a lot of astronomy papers, you will often see the package `emcee` used (Foreman-Mackey et al. 2013). It is an affine-invarient$^†$ ensemble sampler. `emcee` uses an ensemble of walkers that collectively adapt to the posterior geometry, avoiding the need to manually tune a proposal distribution. However, `emcee` is fundamentally a Metropolis-like sampler and does not use gradient information (which we will be using later). This means it struggles in high dimensions (~>10). 

### Tuning
Tuning of the proposal distributions can only be done during burn-in. You cannot tune while collecting final samples, as this would violate the Markov property.

-----------------
*$^†$ don't worry about what this means, we won't be using it in these tutorials. Google it if you are interested*


</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

## Aside: Metropolis-Hastings
Everything we have done so far is the Metropolis algorithm, not Metropolis-Hastings. The difference is small, as the MH algorithm is a generalisation of the Metropolis algorithm. Nevertheless, you will see Metropolis-Hastings talked about everywhere, with little comment on the Metropolis algorithm alone. 

In the Metropolis algorithm, the proposal distribution is symmetric. The probability of proposing $\theta'$ from $\theta$ is the same as the probability of proposing $\theta$ from $\theta'$. The Gaussian proposal we used satisfied this. Metropolis-Hastings generalises the algorithm to allow for a non-symmetric proposal distribution. If the proposal distribution is not symmetric, you would be violating the *detailed balance* condition (see the upcoming section). Thus we need to correct for the asymmetry. We do this by modifying the acceptance ratio, so it becomes:

$$ \alpha = \text{min}\left(1, \frac{\pi(\theta')}{\pi(\theta)} \cdot \frac{q(\theta | \theta')}{q(\theta' | \theta)} \right)$$

The extra factor of $q(\theta | \theta')/q(\theta' | \theta)$ is the Hastings correction. In practice, people just use HMC/NUTS, but you can use asymmetric proposals if you like. 


----------------

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

# Part 5: The Typical Set

We now have a working metropolis sampler, and we have seen how proposal widths affect performance. But before we go onto more modern/advanced algorithms, we need to understand the failure modes of these older algorithms. 

Consider a D-dimensional Gaussian centred on the origin. Where does most of the probability *mass* live? Naively, one might answer at the origin (near the mode), after all, that is where the density is the highest. The issue with this is that the mode is just one point - it takes up essentially no volume. If you care about where the typical sample is drawn from (which you do, if you are fitting a model to data), you need to also consider probability volume. 

In D dimensions, the volume of a thin shell, at radius $r$ scales as $r^{d-1}$. So while the density at the origin is maximal at the origin, there is essentially no volume there.  The density (assuming a gaussian), falls off as $e^{-r^2/2}$. The probability mass is the product of these two effects (volume times density) peaks at a radius of $r = \sqrt{d-1}$. The following figure and caption come from Betancourt 2018:

<div style="text-align: center;">
  <img src="Images/Betancourt2018.PNG" style="max-width: 70%; border-radius: 4px;">
    <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
   In high dimensions a probability density, π(q), will concentrate around its mode, but the volume over which we integrate that density, dq, is much larger away from the mode. Contributions to any expectation are determined by the product of density and volume, π(q) dq, which then concentrates in a nearly-singular neighborhood called the typical set (grey).
  </p>
</div>

This is to say, that the region that contributes most to any expectation$^‡$ is not the mode but at an (increasingly) thin shell around it. This shell is the typical set. Below is a 1D slice of a N-dimensional Gaussian, showing the typical set:

*$^‡$: Every summary statistic that you extract from an MCMC chain (mean, variance, credible intervals, predictable observables) is an expectation.*

</div>

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4))

r = np.linspace(0, 25, 2000)

for ax, d in zip(axes, [2, 10, 100, 500]):
    mass = chi.pdf(r, d)
    
    ax.plot(r, mass, 'k', lw=2)
    ax.axvline(np.sqrt(d-1), ls='--', color='tab:red', 
               label=r'$r = \sqrt{d-1}$' + f' = {np.sqrt(d-1):.1f}')
    ax.axvline(0, ls=':', color='tab:blue', alpha=0.5, label='Mode (r = 0)')
    ax.fill_between(r, mass, alpha=0.15, color='black')
    ax.set_xlabel('Radius r')
    ax.set_ylabel('Posterior mass')
    ax.set_title(f'd = {d}')
    ax.legend(fontsize=9)
    ax.set_xlim(0, max(np.sqrt(d) * 2, 5))

plt.tight_layout()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

## Why typical sets matter for MCMC

The typical set explains the failure modes we saw with Metropolis:
1) When the proposals were too small, the chain explores a tiny neighbourhood and never reaches the typical set.
2) When the proposals were too large, almost every jump lands outside the thin typical set shell, and most proposals are rejected.
3) Goldilocks issue: in higher dimensions, the typical set becomes increasingly thin. When Metropolis needs to propose a new position, it must choose a very specific set of directions that lie on the typical set. However, the number of directions in higher dimensions grows exponentially, so the probability of randomly guessing a good direction falls dramatically.

The below figure (again from Betancourt 2018) demonstrates this issue:

<div style="text-align: center;">
  <img src="Images/Fig10Bentancourt.PNG" style="max-width: 70%; border-radius: 4px;">
    <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
  In high dimensions, the Random Walk Metropolis proposal density (green) is strongly biased towards the outside of the typical set (red) where the target density, and hence the Metropolis acceptance probability vanishes. (a) If the proposal variances are large then the proposals will stray too far away from the typical set and are rejected. (b) Smaller proposal variances stay within the typical set and hence are accepted, but the resulting transition density concentrates tightly around the initial point. Either way we end up with a Markov chain that explores the typical set very, very slowly.
  </p>
</div>


</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

# Part 6: Detailed Balance & Ergodicity

Above, we have claimed that MCMCs will prodice samples from the posterior. For this to work, two (or three, depending on how you count) properties need to be fufilled. These are detailed balance, and ergodicity.

## Detailed Balance

A Markov chain with transition kernel $K(\theta'|\theta)$ satisfies detailed balance with respect to $\pi(\theta)$ if:
$$ \pi(\theta)K(\theta'|\theta) =  \pi(\theta')K(\theta|\theta')$$
In plain English, the probability of being at $\theta$ and transitioning to $\theta'$ equals the probability of the opposite move, going from $\theta'$  to $\theta$.

If detailed balance holds, then $\pi$ is a stationary distribution of the chain. As we have the $\text{min}(1,\cdot)$ in the algorithm as the acceptance ratio, this corrects for the fact that we are preferrentially exploring regions of higher probability mass, satistfying detailed balance. 

## Ergodicity
Detailed balance guarantees that $\pi$ is *a* stationary distribution, but not that the chain will find it. For that, we need to satistify the ergodicity condition: the chain must be irreducible (can reach any state) and aperiodic (no determinisitic cycles). 

For Metropolis with a Gaussian proposal, the Gaussian has support everywhere (i.e. it can propose a point anywhere, there are no points that have probability zero of proposal - this satisfies irreducuiblitlity), and rejection means the chain can stay in place, which allows for aperiodicity.

-------------


</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

# Part 7: Hamiltonian Monte Carlo & NUTS
> A note on scope / complexity: This section covers materials which would take 1-2 lectures to cover properly. You don't need to understand all of the nuts and bolts, but an understanding of why this works is useful. I would highly recommend reading the Betancourt 2018 paper on this, as this largely follows their derivation quite closely.

## Roadmap

| Section | Purpose |
|---------|---------|
| A Vector Field | Introduces the idea of a direction field that follows the typical set |
| The Gradient |  Identifies the gradient as a candidate and shows why it's insufficient |
| The Physical Analogy | Reframes the problem as orbital mechanics, motivating momentum as the missing ingredient |
| Phase Space | Doubles the parameter space to accommodate position and momentum| 
| The Canonical Distribution | Defines a joint distribution that recovers the posterior when momentum is discarded |
| The Hamiltonian | Encodes the geometry of the typical set in a single scalar function (energy) |
| Hamilton's Equations | Derives the vector field — the trajectories — from the Hamiltonian |
| The Hamiltonian-Markov Transition | Assembles the three steps (lift, explore, project) into a working sampler |
| The Euclidean-Gaussian Kinetic Energy |Practical choices: kinetic energy|

The purpose of Hamiltonian Monte Carlo is to be able to follow the typical set in a principled way, as the typical set is the most important region (*the region where it happens*, if you will). 

Recall the typical set, from mere inches above: in higher dimensions, the posterior mass is concentrated in a thin shell. As dimension increases, the volume outside the shell is much larger than the volume inside the shell. Almost every random proposal lands outside the shell, which will likely be rejected. No matter how you tune the covariance matrix of the proposal distribution, Metropolis(-Hastings) (MH) will explore the typical set extremly slowly. 

### A Vector Field

In order to make large jumps away from the initial point, and into new unexplored regions of the typical set, we need to exploit the geomettry of the typical set. Specifically, we need transitions that can follow the contours of high probability mass, coherently gliding through the typical set.

<div style="text-align: center;">
  <img src="Images/Gliding.png" style="max-width: 70%; border-radius: 4px;">
</div>

What we need is for our walkers to be able to follow a sign-posted path pointing them in the right direction. To put it more precisely, a vector field aligned with the typical set would be very helpful. By construction, this would move us into a new point in the typical set. We need a way to create this vector field from the information contained within the target distribution. 

<div style="text-align: center;">
  <img src="Images/Spiral.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
  We want something that looks like this.
  </p>
</div>


## The Gradient
The natural information that we have not exploited yet is the gradient of the target density. The gradient defines a vector field in parameter space that is sensitive to the geometry of the target distribution. This seems promising, but there is a pretty big issue - the gradient points towards the mode of the distribution, and away from the typical set. 

This is a hurdle. Can you think of any situation of a vector field pointing to a single point, but there is a way to use that vector field to draw paths that don't follow the field?

<div style="text-align: center;">
  <img src="Images/Orbit.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
  Perhaps something like this?
  </p>
</div>

To utilise the information in the gradient, we need to complement it with additional geometric constraints that twist the gradient direction to align with the typical set. We need to add **momentum**. Fortunately there is a way to do this, rooted in differential geometry.

## The Physical Analogy
For reasons rooted in the mathematics of differential geometry (which we won't derive here, because I don't understand it), for every probabilistic system there is a mathematically equivalent physical system that we can reason about more easily. Instead of thinking about a mode, a gradient, and a typical set, we can *equivalently* reason about a planet, a gravitational field, and an orbit. I imagine that if you are an astronomer, this could be helpful.

If we place a satellite at rest in the gravitational field, it falls and crahses into the surface of the planet, just as a naive gradient-driven trajectory would crash into the mode. 

But using the physical picture, we can immediately see a solution: we can give the satelitte enough momentum to counteract the gravitational field and maintain a stable orbit. Too little momentum and it will spiral inward, too much and it flies off. But with enough momentum, the gravitational force and the dynamics are conservative.

Hence the key to twisting the gradient vector field into a vector field aligned with the typical set is to expand our probabilistic system with the introduction of auxiliary momentum parameters. There is only one correct way to do this, and it leads directly to Hamiltonian Monte Carlo.

## Phase Space
Conservative dynamics$^♣$ in physical systems require that volumes are preserved. As the system evolves, any compression in position space must be compensated by a corresponding expansion in momentum space, and vice versa. This means that if we change our parameterisation of parameter space (say working in sin of a parameter rather than the parameter directly), we need to be able to change momentum space so that the parameter-momentum space volume is preserved. To mimic this, we introduce auxiliary momentum parameters $p_n$ to complement each dimension of the target parameter space:

$$\theta_n \rightarrow (\theta_n, p_n) $$

This expands the D-dimensional parameter space into a 2D-dimensional *phase space*. Note we are not adding a single extra dimension, we are doubling the number of dimensions. Each parameter gets a momentum parameter. 

--------
*Note: from here on we will use $q$ rather than $\theta$ for our parameters, following the convention of Hamiltonian mechanics where $q$ denotes co-ordinates and $p$ denotes momenta. They are the same thing: $q\equiv\theta$*

*Also note: in the following we will use the notation $\pi(q) \equiv \pi(\theta)$ to represent a generic probability distribution function. This is not the prior, and it would be better thought of as the posterior distribution function.*

*$^♣$: A system has conservative dynamics when its total energy is constant along any trajectory — nothing is gained or lost, only exchanged between kinetic and potential forms. Examples include pendulums and orbits. A consequence of energy conservation is that the dynamics preserve volumes in phase space. Liouville's theorem: if you track a cloud of initial conditions forward in time, the cloud may deform in shape but its total volume remains constant. This volume preservation is what guarantees that the dynamics don't systematically concentrate or disperse probability.*

----------

## The Canonical Distribution
Now that we have expanded to phase space, we need to define a *joint-*probability distribution over both position and momentum. We can't just use $\pi(q)$ as it says nothing about $p$. And we can't pick an arbitrary joint distribution, because we need the momentum to be auxiliary: when we are done exploring, we throw it away and must be left with samples from the posterior $\pi(q)$.

The solution is to construct the joint distribution as a product:

$$\pi(q, p) = \pi(p|q) \pi(q) $$
This is called the canonical distribution. The first factor, $\pi(p|q)$ is a conditional distribution over momentum that we get to choose (more on this shortly). The second factor is the target posterior that we have been talking about this whole time. Written this way, it is immediate from the rules of probability that we can marginalise out the momentum to recover the target:

$$\int \pi(q, p) dp = \pi(q)\cdot \int \pi(q| p)dp = \pi(q)  $$

This is to say that if we can build a sampler that explores the joint typical set of $\pi(q, p)$, then simply discarding the $p$ coordinates gives us samples from $\pi(q)$, the posterior that we care about. It is kind of like multiplying by one or adding zero in a sense. 

<div style="text-align: center;">
  <img src="Images/Canon.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
  By constructing a probability distribution on phase space that marginalizes to the target distribution, we ensure that the typical set on phase space projects to the typical set of the target distribution. In particular, if we can construct trajectories that efficiently explore the joint distribution (red arrow) they will project to trajectories that efficiently explore the target distribution (green arrow)
  </p>
</div>

## The Hamiltonian
Because the canonical density does not depend on a particular choice of parameterisation, we can write it in terms of an invariant function called the Hamiltonian:

$$\pi(q,p) = e^{-H(q,p)} $$
inverting:
$$H(q,p)  = -\log \pi(q,p)$$

Appealing to the physical analogy, the value of the Hamiltonian at any point in phase space is called the energy at that point. 

Becuase of the decompositionof the joint density, the Hamiltonian itself decomposes into two terms:
$$\begin{aligned}
H(q,p) &= -\log\pi(p|q) - \log\pi(q) \\
&\equiv K(p,q) + V(q)
\end{aligned}$$

Here:
- $V(q)$ is the potential energy and is completly determined by the target distribution (the posterior). This is the negative log-posterior, and it encodes the landscape the sampler must navigate.
- $K(p,q) is the kinetic energy, our choice of momentum distribution. This is unconstrained and must be specified by the implementation.

## Hamiltonians Equations
Because the Hamiltonian captures the geometry of the typical set, we should be able to use it to generate a vector field oriented with the typical set. Indeed, we can use Hamilton's Equations: 

$$\begin{aligned} \frac{dq}{dt} &= +\frac{\partial H}{\partial p} = \frac{\partial K}{\partial p} \\ \frac{dp}{dt} &= -\frac{\partial H}{\partial q} = -\frac{\partial K}{\partial q} - \frac{\partial V}{\partial q} \end{aligned}$$

Note that $\partial V/\partial q$ is the gradient of the negative log-posterior which is exactly the gradient information we wanted to exploit. The equations split the gradient force (which points towards the mode/the planet) across two coupled equations. The gradient changes the momentum, not the position (directly) and the momentum changes the position. The position update $dq/dt$ moves you in the direction of the current momentum, not in the direction of the gradient. The gradient only deflects the trajectory. So these equations do twist the gradient into a vector field that follows the typical set rather than falling towards the mode. 

Following the Hamiltonian vector field for some time $t$ generates trajectories $\phi_t(q,p)$ that rapidly move through phase space while being constrained by the typical set. Projecting these trajectories back down onto the target parameter space finally yields the efficient exploration of the typical set that we have been searching for.

<div style="text-align: center;">
  <img src="Images/PhaseSpace.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
         Every Hamiltonian Markov transition is comprised of a random lift from the target parameter space onto phase space (light red), a deterministic Hamiltonian trajectory through phase space (dark red), and a projection back down to the target parameter space (light red).
  </p>
</div>

## The Hamiltonian-Markov Transition

In order to utilise these Hamiltonian trajectories to construct an efficient Markov transition, we need three steps:

1) Step 1: **Lift**. To lift an initial point in parameter space into one in phase space, we *sample* momentum from the conditional distribution: $$p\sim\pi(p|q) $$ If the intial point was in the typical set of the target of the target distribution, sampling the momentum directly from its conditional distribution guarantees that the lifted point falls into the typical set of the joint phase-space distribution.
2) Step 2: **Explore**. Once in phase space, we integrate Hamilton's equations for some time: $$(q,p) \rightarrow \phi_t(q,p)$$ By construction, these trajectories coherently push the transition away from the initial point while staying confined to the joint typical set.
3) Step 3: **Project**. After integrating, we return to the parameter space by discarding the momentum: $$(q,p)\rightarrow(q) $$

Composing these three steps yields a Hamiltonian Markov transition composed of random trajectories that rapidly explore the target distribution. Out in the tails, the momentum sampling and projection steps allow the chan to rapidly fall toward the typical set. Once the chain has reached the typical set, the Hamiltonian trajectories ensure extremely efficient exploration.

<div style="text-align: center;">
  <img src="Images/Jumper.png" style="max-width: 70%; border-radius: 4px;">
     <p style="font-size: 0.75em; color: #888; font-style: italic; margin-top: 0.5em; text-align: center;">
         (a) Each Hamiltonian Markov transition lifts the initial state onto a random level set of the Hamiltonian, which can then be explored with a Hamiltonian trajectory before projecting back down to the target parameter space. (b) If we consider the projection and random lift steps as a single momentum resampling step, then the Hamiltonian Markov chain alternates between deterministic trajectories along these level sets (dark red) and a random walk across the level sets (light red).
  </p>
</div>

## The Euclidean-Gaussian Kinetic Energy
The kinetic energy $K$ (i.e. the choice of momentum distribution) is a free parameter. The standard choice is a Gaussian:

$$\pi(p|q) = \mathcal{N}(p|0, M)$$

In plain language: the momentum $p$ is drawn from a multivariate Gaussian distribution centred on zero with a covariance matrix M. Recall that $K = -\log\pi(p|q)$, which gives the kinetic energy:

$$K(p) = \frac{1}{2}p^T M^{-1}p + \text{const.} $$

The matrix M is called the mass matrix (in the physical analogy), or the inverse Euclidean metric in the geometric picture. The choice of M effectively rotates and rescales parameter space. As $M^{-1}$ more closely resembles the covariance of the target distribution, the posterior becomes more isotropic in the transformed space, making the energy levels more uniform and hence easier to explore. If you are familiar with General Relativity, this may sound familiar. M is essentially the metric tensor on parameter space. 

The optimal choice of $M^{-1} = \text{Cov}_\pi (\theta)$, which is the posterior covariance itself. This is a bit of a chicken and the egg situation, but in practice this gets repeatedly estimated during warmup.

As one might expect, it is often hard to analytical solve Hamiltons equations. Instead numerical integrators, such as leapfrog, are used. See the Betancourt 2018 paper for details. 

### NUTS: The No-U-Turn Sampler

HMC has two tuning parameters beyond the mass matrix: the step size $\varepsilon$ and the number of leapfrog steps $L$ (equivalently, the total integration time $T = L\varepsilon$).

The step size $\varepsilon$ is adapted automatically during warmup by targeting a specific acceptance probability (typically 0.8). But $L$ is harder to set. Too few steps and you don't move far from the start, you get the same slow diffusion as Metropolis. Too many steps and the trajectory curves back toward the start, wasting computation. The No-U-Turn Sampler (NUTS; Hoffman & Gelman 2014) eliminates $L$ as a tuning parameter entirely. It grows the trajectory in both directions (forward and backward in time) until it detects a U-turn. It then samples a point from the completed trajectory. 

## Summary

To recap the full picture: at each iteration, HMC draws a random momentum (setting a random direction and speed), then simulates a physical trajectory through parameter space using Hamilton's equations, and finally proposes the endpoint as the next sample. The trajectory is computed numerically via the leapfrog integrator, and a Metropolis correction ensures exactness despite integration error. The mass matrix $M$ rescales the posterior geometry so the sampler moves efficiently in all directions, and NUTS automates the trajectory length by stopping when a U-turn is detected. The result is a sampler that makes large, directed moves through the typical set (in contrast to the small, random, diffusive steps of Metropolis) while maintaining the detailed balance and ergodicity guarantees that make it a valid MCMC algorithm.

It is basically the G.O.A.T. of MCMC algorithms.
</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

# Part 8: NumPyro Demonstration

In Parts 4–7 we built MCMC from scratch and developed the theory behind HMC. In practice, you will never implement HMC yourself. You will use a probabilistic programming library that does it all for you. We will use `NumPyro`. 

NumPyro is a lightweight probabilistic programming library built on top of `JAX`. `JAX` provides automatic differentiation (needed for the gradient $\partial V/ \partial q$) and just-in-time compilation. 

Before we begin, let's check that everything is installed.

</div>

In [ ]:
install_ok = True

try:
    import jax
    print(f"  ✓ JAX {jax.version}")
except ImportError:
    print("  ✗ JAX not installed. Run: pip install jax jaxlib")
    install_ok = False
try:
    import jax.numpy as jnp
    print(f"  ✓ jax.numpy")
except ImportError:
    print("  ✗ jax.numpy not available")
    install_ok = False
try:
    import numpyro
    print(f"  ✓ NumPyro {numpyro.version}")
except ImportError:
    print("  ✗ NumPyro not installed. Run: pip install numpyro")
    install_ok = False
try:
    import numpyro.distributions as dist
    from numpyro.infer import MCMC, NUTS, Predictive
    from numpyro.diagnostics import print_summary
    from jax import random
    print(f"  ✓ numpyro.infer, numpyro.distributions, numpyro.diagnostics")
except ImportError:
    print("  ✗ NumPyro submodules not available")
    install_ok = False
try:
    import arviz as az
    print(f"  ✓ ArviZ")
except ImportError:
    print("  ✗ ArviZ not installed. Run: pip install arviz")
    install_ok = False
try:
    import corner
    print(f"  ✓ corner")
except ImportError:
    print("  ✗ corner not installed. Run: pip install corner")
    install_ok = False
try:
    import matplotlib.pyplot as plt
    print(f"  ✓ matplotlib")
except ImportError:
    print("  ✗ matplotlib not installed. Run: pip install matplotlib")
    install_ok = False
try:
    import numpy as np
    print(f"  ✓ numpy {np.version}")
except ImportError:
    print("  ✗ numpy not installed")
    install_ok = False
if install_ok:
    numpyro.set_platform("cpu")
    print("\n  All packages found. You're good to go.")
else:
    print("\n  ⚠️ Some packages are missing. Install them before continuing.")

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

## How NumPyro Models Work

A NumPyro model is just a Python function. Inside that function, you use three key primitives:

- **`numpyro.sample("name", distribution)`** — declares a random variable with a prior distribution. This tells NumPyro: "this parameter exists, and before seeing data, I believe it follows this distribution."

- **`numpyro.sample("name", distribution, obs=data)`** — declares an observed random variable. This is the likelihood: it says "the data was generated from this distribution, given the current parameter values." The `obs=data` keyword is what distinguishes a prior from a likelihood in NumPyro.

- **`numpyro.factor("name", log_value)`** — adds an arbitrary log-density contribution to the model's log-probability, without declaring a new random variable. This is useful for custom likelihoods that don't correspond to a built-in distribution. You can think of `numpyro.sample("obs", dist, obs=data)` as syntactic sugar that internally calls `numpyro.factor` with the log-probability of `data` under `dist`.

That's it. From these three primitives, NumPyro automatically constructs the log-posterior $\log[\mathcal{L}(\theta)\pi(\theta)]$, computes its gradient via JAX autodiff, and runs NUTS.

## A Minimal Example

Let's fit a coin. We observe 7 heads out of 10 flips and want to infer the bias $\theta$:

</div>


In [ ]:
def coin_model(data):
    # Prior: theta is uniform on [0, 1]
    theta = numpyro.sample("theta", dist.Uniform(0, 1))
    # Likelihood: each flip is Bernoulli with probability theta
    numpyro.sample("obs", dist.Bernoulli(theta), obs=data)

# Data: 7 heads (1) and 3 tails (0)
data = jnp.array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1])

# Set up NUTS and run
kernel = NUTS(coin_model)
mcmc = MCMC(kernel, num_warmup=500, num_samples=2000, num_chains=4)
mcmc.run(random.PRNGKey(0), data)
mcmc.print_summary()


<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
Let's unpack the inference lines:

- `NUTS(coin_model)` — wraps your model in the NUTS sampler. This is where all of Part 7 lives: the leapfrog integrator, the mass matrix, the U-turn detection.
- `MCMC(kernel, num_warmup=500, num_samples=2000, num_chains=4)` — sets up the MCMC run. `num_warmup` is the number of steps used for adaptation (step size, mass matrix) — these samples are discarded. `num_samples` is the number of samples to keep per chain. num_chains runs 4 independent chains for diagnostic purposes.
- `random.PRNGKey(0)` — JAX requires explicit random seeds. This ensures reproducibility.
- `mcmc.print_summary()` — prints posterior summaries and diagnostics.

The columns to pay attention to:

- n_eff: effective sample size. You have 8000 total samples (2000 × 4 chains), but autocorrelation reduces this. Here n_eff ≈ 2500, which is plenty.
- r_hat: the Gelman-Rubin diagnostic. Should be ~1.00. Values > 1.01 indicate the chains disagree, and you should not trust the results.
- Number of divergences (printed separately): should be 0.

</div>

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">


## Fitting the RV Curve with `NumPyro`

*NOTE: This will require some tuning to get it aligned with the posterior from above. Good luck!*


Now let's refit the radial velocity curve from Part 4, when we used Metropolis, but now using HMC/NUTS.

### Model
$$ v_r(t) = \gamma + K \sin \left(\frac{2\pi t}{P} + \phi \right) $$

We will set the true value and (simple) priors:

| Parameter | Symbol | True Value | Prior | Notes |
|-----------|:------:|------------|-------|-------|
| System velocity | $\gamma$ | 5 m/s | Uniform(-50, 50) | |
| Semi-amplitude  | K | 2 m/s | Uniform(0, 100) | Must be positive |
| Period | P | 30 days | Uniform(0.1, 200) | |
| Phase | $\phi$ | 0.5 rad | Uniform(0, $2\pi$) | |


</div>

In [ ]:
def rv_model(t, v_obs, v_err):
    """
    NumPyro model for a single-planet circular RV orbit.Parameters (all inferred):
    gamma : systemic velocity [m/s]
    K     : semi-amplitude [m/s]
    P     : orbital period [days]
    phi   : phase offset [rad]
    """
    gamma = numpyro.sample("gamma", dist.Uniform(-50, 50))
    K     = numpyro.sample("K",     dist.Uniform(0, 100))
    P     = numpyro.sample("P",     dist.Uniform(1, 200))
    phi   = numpyro.sample("phi",   dist.Uniform(0, 2 * jnp.pi))
    
    # Forward model
    model = gamma + K * jnp.sin(2 * jnp.pi * t / P + phi)
    
    # Likelihood: observed velocities are Gaussian-distributed around the model
    numpyro.sample("obs", dist.Normal(model, v_err), obs=v_obs)

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">

Note the structure: four `numpyro.sample` calls declare the priors, then one `numpyro.sample(..., obs=...)` call declares the likelihood. The forward model in between is just numpy/jax — NumPyro doesn't care what you do between the prior declarations and the likelihood, as long as it's differentiable (which basic arithmetic and trig functions are).

Now run it:

</div>


In [ ]:
np.random.seed(42)

# Get observations
n_obs = 15
t_obs = np.sort(np.random.uniform(0, 100, n_obs))

# STD of Uncertainties, ~1m/s with some spread (differing SNR on different nights)
v_err = np.random.uniform(0.8, 2.5, n_obs)

# True Parameters
gamma_true = 5   # m/s
K_true     = 2  # m/s
P_true     = 30  # days
phi_true   = 0.5 # rads

# Generate data
v_model = gamma_true + K_true * np.sin(2*np.pi*t_obs/P_true + phi_true)
v_obs   = v_model + v_err * np.random.randn(n_obs) # realisation of noise

# Run NUTS (t_obs, v_obs, v_err are defined from Part 4)
kernel = NUTS(rv_model)
mcmc = MCMC(kernel, num_warmup=500, num_samples=2000, num_chains=8)
mcmc.run(random.PRNGKey(161), jnp.array(t_obs), jnp.array(v_obs), jnp.array(v_err))

mcmc.print_summary()

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
    
The arguments after the random key are passed directly to your model function. NumPyro matches them positionally: `t` gets `t_obs`, `v_obs` gets `v_obs`, `v_err` gets `v_err`.
    
### Extracting and Plotting Samples
`mcmc.get_samples()` returns a dictionary mapping parameter names to arrays of posterior samples (concatenated across chains):
</div>

In [ ]:
samples = mcmc.get_samples()
# samples is a dict: {'gamma': array, 'K': array, 'P': array, 'phi': array}
# Each array has shape (num_samples * num_chains,) = (8000,)

print(f"Posterior mean of K: {np.mean(samples['K']):.2f} m/s")
print(f"Posterior std of K:  {np.std(samples['K']):.2f} m/s")

# Corner plot
sample_array = np.column_stack([
    np.array(samples['gamma']),
    np.array(samples['K']),
    np.array(samples['P']),
    np.array(samples['phi']),
])

labels = [r'$\gamma$ [m/s]', r'$K$ [m/s]', r'$P$ [days]', r'$\phi$ [rad]']
truths = [gamma_true, K_true, P_true, phi_true]

fig = corner.corner(sample_array, labels=labels, truths=truths,
                    truth_color='black', show_titles=True,
                    title_kwargs={"fontsize": 12})

<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
    
Compare the corner plot to the one you made with Metropolis in Part 4. You should see a similar posterior, but the NUTS samples will be much less correlated and the effective sample size should be dramatically higher for the same number of total samples. If you are getting a different posterior, try changing some of the priors, or even reparameterising in freqeuncy rather than using period.

### Posterior Predictive Plot
Finally, let's draw random posterior samples and plot the corresponding model curves over the data. This is a posterior predictive check: if the model is correct and the inference worked, the posterior curves should bracket the data.
</div>

In [ ]:
t_fine = np.linspace(0, 100, 500)

#Draw 100 random posterior samples
idx = np.random.choice(len(sample_array), 100, replace=False)


fig, axes = plt.subplots(1, 2, figsize=(12, 4),
gridspec_kw={'width_ratios': [2, 1]})

#Panel 1: Time series
ax = axes[0]
for i in idx:
    g, K, P, phi = sample_array[i]
    ax.plot(t_fine, g + K * np.sin(2 * np.pi * t_fine / P + phi),
    color='black', lw=0.3, alpha=0.15)
    
ax.plot([], [], color='black', lw=1, alpha=0.5, label='Posterior samples')
ax.plot(t_fine, gamma_true + K_true * np.sin(2 * np.pi * t_fine / P_true + phi_true),
color='tab:blue', lw=1.5, alpha=0.7, label='True model')
ax.errorbar(t_obs, v_obs, yerr=v_err, fmt='o', color='k',
ecolor='grey', capsize=2, ms=5, label='Observations', zorder=5)
ax.set_xlabel('Time [days]')
ax.set_ylabel('Radial velocity [m/s]')
ax.legend(fontsize=9)
ax.set_title('Posterior Predictive: Time Series')

#Panel 2: Phase-folded
ax = axes[1]
phase_fine = np.linspace(0, 1, 500)
phase_obs = ((t_obs / P_true) % 1)
for i in idx:
    g, K, P, phi = sample_array[i]
    ax.plot(phase_fine, g + K * np.sin(2 * np.pi * phase_fine + phi),
    color='tab:orange', lw=0.3, alpha=0.15)
    
ax.plot(phase_fine,
gamma_true + K_true * np.sin(2 * np.pi * phase_fine + phi_true),
color='tab:blue', lw=1.5, alpha=0.7)
ax.errorbar(phase_obs, v_obs, yerr=v_err, fmt='o', color='k',
ecolor='grey', capsize=2, ms=5, zorder=5)
ax.set_xlabel('Phase')
ax.set_ylabel('Radial velocity [m/s]')
ax.set_title('Phase-folded')
plt.tight_layout()
plt.show()


<div style="max-width: 800px; margin: 0 auto; text-align: justify;">
    
-----------

## Further Reading

- **Betancourt (2018)**, "A Conceptual Introduction to Hamiltonian Monte Carlo" — §1–2: Typical set. §3: Hamiltonian dynamics. §4–5: Implementation. §6: Diagnostics.
- **Hogg & Foreman-Mackey (2018)**, "Data Analysis Recipes: Using Markov Chain Monte Carlo" — §3: Metropolis. §5: Convergence. §6: Tuning. §9: Troubleshooting.
- **Sharma (2017)**, "MCMC Methods for Bayesian Data Analysis in Astronomy" — §3.7: Convergence. §3.10: HMC.
- **Foreman-Mackey et al. (2013)**, "emcee: The MCMC Hammer"
- **Hoffman & Gelman (2014)**, "The No-U-Turn Sampler"

-----------
 </div>